In [ ]:
from keras.models import Sequential
from keras import optimizers
from keras.layers import Dense, Activation, Flatten, Conv2D, MaxPooling2D, Dropout
import random

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%cd /content/drive/MyDrive/AI_teamproject/AI_Teamproject/

In [ ]:
import numpy as np

x = np.load('./covid_19_data/x.npy')
y_data= np.load('./covid_19_data/y.npy')

y=np.array([[0,0,0]])
for i in y_data:
    if i == 0:
        label=np.array([1,0,0])
    elif i == 1:
        label=np.array([0,1,0]) 
    else:
        label=np.array([0,0,1])  
    label=label.reshape(1,3)
    y=np.concatenate([y,label],axis=0)
# 이게 one-hot vector 값 ( 처음에 초기화 한거 지움 )
y=np.delete(y,0,axis=0)



In [ ]:
test_idx = random.sample(range(len(x)),int(0.3*len(x)))
x_test = x[test_idx,:,:]
x_train = np.delete(x, test_idx, axis=0)
y_test = y[test_idx,:]
y_train = np.delete(y, test_idx, axis=0)


x_train = x_train.reshape(len(x_train), 224,224,1)
x_test = x_test.reshape(len(x_test), 224,224,1)


import cupy as np
x_train = np.array(x_train)
y_train = np.array(y_train)
x_test = np.array(x_test)
y_test = np.array(y_test)


In [ ]:
x_train.shape

In [ ]:
model = Sequential()
model.add(Conv2D(input_shape = (224,224,1), filters = 64, kernel_size = (3,3), strides = (1,1), padding = 'same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size = (2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(input_shape = (112,112,64), filters = 128, kernel_size = (3,3), strides = (1,1), padding = 'same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size = (2,2)))
model.add(Dropout(0.25))

model.add(Flatten())

# final layer with 10 neurons to classify the instances
model.add(Dense(3, activation = 'softmax'))

adam = optimizers.Adam(lr = 0.003)

model.compile(loss = 'categorical_crossentropy', optimizer = adam, metrics = ['accuracy'])

In [ ]:
# model summary 한 것
model.summary()

In [ ]:
history = model.fit(x_train,y_train, batch_size = 63, validation_split = 0.2, epochs = 20, verbose = 0)


import matplotlib.pyplot as plt

plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.legend(['training', 'validation'], loc = 'upper left')
plt.show()

# 각종 값 출력
print(hist.history['loss'])
print(hist.history['accuracy'])
print(hist.history['val_loss'])
print(hist.history['val_accuracy'])

In [ ]:
results = model.evaluate(x_test, y_test)
print('Test accuracy: ', results[1])